# Membership Inference Attacks Against Machine Learning Models

Elements of the adversarial system:
* Shadow model
* Attack model

###  Shadow Model

The adversary trains $k$ shadow models, each on a dataset (shadow data) that is similar
in format and distribution as the __target model's private training set__.

Generate shadow data drawing samples from the population where the target model's training data
are drawn from. The shadow model must be __trained in a similar fashion as the target model__.
The larger the number of shadow models, the better the accuracy of the attack model.

### Attack Model

The adversary queries each shadow model with its own disjoint training and test dataset which are of the __same size__.

# Generating training data for shadow models

The adversary needs training data that is distributed similarly to the target model's training data. 
Shoki's proposed methods are:

## Model-based synthesis:
Attacker does not have real raining data nor any statistics abouts its distribution.
Generate synthetic data for the shadow model using the target model itself. The intuition is that records that are classified by the target model 
with high confidence should be statistically similar to the target's training training dataset.

Two phases of synthesis process:
1. _Search_ using a hill-climbing algorithm the space of possible data records to find inputs that are classified by the target model with high confidence.
2. _Sample_ synthetic data from records. After synthetised a record, repeat until shadow dataset is full

>>
First fix class $c$ for which attacker wants to generate synthetic data.
Initialize randomly a data record $\mathbf{x}$.
The attacker must known the syntactic format of data records (number of features and numerical range of each feature).
Sample the value for each feature uniformly at random from among all possible values of that feature.
>>
A proposed record is _accepted_ only if it increases the [hill-climbing](https://en.wikipedia.org/wiki/Hill_climbing) objective: the probability of
being classified by the target model as class $c$.
>>
Each iteration involves proposing a new candidate record by changing $k$ randomly selected features of the latest accepted record $\mathbf{x^∗}$.
This is done by flipping binary features or resampling new values for features of other types.
We initialize $k$ to $k_{max}$ and divide it by 2 when $rej_{max}$ subsequent proposals are rejected. This controls the diameter of search around the
accepted record in order to propose a new record. We set the
minimum value of $k$ to $k_{min}$.
This controls the speed of the search for new records with a potentially higher classification
probability $y_c$.
>>
The second, sampling phase starts when the target model’s
probability $y_c$ that the proposed data record is classified as belonging to class $c$ is larger than the probabilities for all
other classes and also larger than a threshold $conf_{min}$.
This ensures that the predicted label for the record is $c$, and that the target model is sufficiently confident in its label prediction. We
select such record for the synthetic dataset with probability $y_{c}^∗$
and, if selection fails, repeat until a record is selected.
>>
This synthesis procedure works only if the adversary can
efficiently explore the space of possible inputs and discover
inputs that are classified by the target model with high confi-
dence. For example, it may not work if the inputs are high-
resolution images and the target model performs a complex
image classification task.


#### Example with iris data:

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler

get data, normalize and fit RF model

In [2]:
rf = RandomForestClassifier(n_estimators=100)
data, target = load_iris(return_X_y=True)

In [3]:
scaler = MinMaxScaler()
data_std = scaler.fit_transform(data)

In [4]:
rf.fit(data_std, target);

Check if synthesized data belongs to the fixed class 

In [5]:
%run ../data_synthesis.py

In [6]:
# class we want the record to belong to
fixed_class = 2
# synth record
x = synthesize(rf, fixed_class=fixed_class, k_max=3)
# check if synth record is classified correctly
assert fixed_class == np.argmax(rf.predict_proba(x))

Now check if batch mode works

In [7]:
x = synthesize_batch(rf, fixed_class, n_records=100)
assert all(np.argmax(rf.predict_proba(x), axis=1) == fixed_class)

---

## Statistics-based synthesis

>>
The attacker may have some statistical information about the population from which the target
model’s training data was drawn.
For example, the attacker may have prior knowledge of the marginal distributions of different features.
In our experiments, we generate synthetic training records for the shadow models by independently
sampling the value of each feature from its own marginal distribution.
The resulting attack models are very effective.